In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt

from holodeck import plot, detstats, utils
from holodeck.constants import YR, MSOL, MPC, GYR
import holodeck as holo
from holodeck import extensions as holo_extensions


# Make realizer

In [ ]:
f_cents, f_bins = holo.utils.pta_freqs()
realizer = holo_extensions.Realizer(f_bins/2, resample=2.0, lifetime=3*GYR, mamp=1e9*MSOL, dens=2.0)


In [ ]:
print(realizer)

In [ ]:
for key in realizer.__dict__.keys():
    print(key)

binary_vals = realizer._binary_vals
vals_names = realizer._vals_names
weights = realizer._binary_weights

print(f"{vals_names=}")
print(f"{binary_vals[0].shape=}")
print(f"{len(realizer._binary_vals)=}")
# print(f"{realizer._binary_weights=}")
print(f"{weights.shape=}")

for ii in range(4):
    print(f"{realizer._vals_names[ii]} vals:", holo.utils.stats(realizer._binary_vals[ii]))

In [ ]:
for ii, val in enumerate(vals_names):
    # if ii==1 or ii==3:
    #     xscale='linear'
    # else:
    xscale='log'
    fig, ax = plot.figax(xlabel=val, ylabel='weights', xscale=xscale)
    ax.scatter(10**binary_vals[ii], weights, s=1, alpha=0.1)
    print(f"{realizer._vals_names[ii]} vals:", holo.utils.stats(realizer._binary_vals[ii]))

# Calling realizer()

realizer() returns names, samples

names is just ['mtot', 'mrat', 'redz', 'fobs']


* mtots = 10^samples[0]
* mrats = 10^samples[1]
* redz = 10^samples[2]
* f0 = 10^samples[3]

but we want to keep all in log space

In [ ]:
names, samples = realizer(down_sample=50)

In [ ]:
print(samples)

In [ ]:
print(samples.shape)

# Make SAM version

In [ ]:
params = {'hard_time': 2.3580737294474514, 
          'gsmf_phi0': -2.012540540307903, 
          'gsmf_mchar0_log10': 11.358074707612774, 
          'mmb_mamp_log10': 8.87144417474846, 
          'mmb_scatter_dex': 0.027976545572248435, 
          'hard_gamma_inner': -0.38268820924239666}

In [ ]:
shape = [10,5,15]

pspace = holo.param_spaces.PS_Uniform_09B(holo.log, nsamples=1, sam_shape=shape, seed=None)
sam, hard = pspace.model_for_params(params=params, sam_shape=shape)

In [ ]:
NREALS = 15
NLOUDEST = 100

data = holo.librarian.run_model(sam, hard, nreals=NREALS, nloudest=NLOUDEST, nfreqs=40,
                              singles_flag=True, params_flag=True)

In [ ]:
# properties in log space, for samples

fobs = np.log10(data['fobs_cents'])
mtot = np.log10(data['sspar'][0]) # log g
mrat = np.log10(data['sspar'][1]) 
redz = np.log10(data['sspar'][3]) # final redshift, not initial

In [ ]:
for arr in [mtot, mrat, redz, fobs]:
    print(arr.shape)

In [ ]:
def realizer_sam(params, nreals, nloudest, nfreqs=40,
               pspace = holo.param_spaces.PS_Uniform_09B(holo.log, nsamples=1, sam_shape=None, seed=None)):
    """ Like Realizer but for SAMs instead of illustris populations

    Parameters
    ----------

    Returns
    -------
    names : names of parameters
    samples : [R, 4, nloudest*nreals] NDarray
        mtot, mrat, redz, and fobs of each source 

    sspar is in shape [F,R,L]
    
    """
    
    sam, hard = pspace.model_for_params(params=params, sam_shape=None,)
    data = holo.librarian.run_model(sam, hard, nreals=nreals, nloudest=nloudest, nfreqs=nfreqs,
                              singles_flag=True, params_flag=True)
    
    vals_names=['mtot', 'mrat', 'redz', 'fobs']
    fobs = np.log10(data['fobs_cents'])
    fobs = np.repeat(fobs, nreals*nloudest).reshape(nfreqs, nreals, nloudest)
    mtot = data['sspar'][0] # g
    mrat = data['sspar'][1]
    redz = data['sspar'][3] # final redshift, not initial

    samples = []
    for par in [mtot, mrat, redz, fobs]: # starts in shape F,R,L
        par = np.swapaxes(par, 0, 1) # R,F,L
        par = par.reshape(nreals, nfreqs*nloudest) # R, F*L
        samples.append(np.log10(par))
    samples = np.array(samples) # 4,R,F*L
    samples = np.swapaxes(samples, 0,1) # R,4,F*L

    return vals_names, samples

In [ ]:
vals_names, samples = realizer_sam(params, 1, 100, pspace=pspace)
print(f"{samples.shape=}")
samples = samples.squeeze()
print(f"{samples.shape=}")

# Try Realizer_SAM class

In [ ]:
print(sam.shape)

In [ ]:
shape = None
realizer = holo_extensions.Realizer_SAM(
    fobs_orb_edges=f_bins/2, 
    params=params,
    pspace=holo.param_spaces.PS_Uniform_09B(holo.log, nsamples=0, sam_shape=shape, seed=None))

In [ ]:
names, samples, weights = realizer(nreals=3, clean=True)

In [ ]:
for rr in range(len(weights)):

    print(f"{weights[rr].shape=}")
    print(f"{samples[rr][0].shape=}")

In [ ]:
names, samples2, weights2 = realizer(nreals=3, clean=False)

In [ ]:
print(f"{weights2.shape=}")
print(f"{samples2[0].shape=}")

In [ ]:
print(np.sum(weights[:,2]==0))

In [ ]:
test = np.arange(20)
test[test%3==0] = 0